# Problem Solving Agents

A **problem-solving agent** uses **atomic** representations - states of the world are considered as wholes, without internal structure - to **search** for a solution.

There are two main families of such algorithms:

- **Uninformed**: given the problem definition, will find a solution
- **Informed**: need more guidance to solve a problem

## Goals and Problems

With **goals** we help organize the behaviour by limiting the objectives the agent is trying to achieve. **Goal formulation** is the first step in problem solving.

**Problem formulation** is the process of deciding what actions and states to consider.

Before proceeding we have to list our assumptions about the **environment**:

- *Observable* -> the agent always knows the current state
- *Discrete* -> in any given state there is only a finite number of actions
- *Known* -> the agent knows which states are reached by each action
- *Deterministic* -> each action has exactly one outcome

### Under these assumptions the solution to any problem is a fixed sequence of actions.

> $Problem->Sequence \: of \: actions->Solution$

This process of looking for a sequence of actions that reaches the goal is called **search**. A search algorithm takes a problem as input and returns a solution in the form of an action sequence.

## Define Problems and Solutions

A problem can be defined formally by 5 components:

- **Initial state** -> The state the agent starts in
- **Actions** -> A description of what an agent can do
- **Transition model** -> Description of what each action does
- **Goal test** -> Determines whether a given state is a goal state
- **Path cost** -> Function that assigns a numeric cost to each path

***

# Searching for Solutions

The possible action sequences starting at the initial state form a **search tree** with the initial state at the root. The branches are actions and the nodes correspond to states in the state space of the problem.

We start at the root node by expanding the current state by applying each legal action to the current state. In this way we generate a new set of states. This is the essence of search: following up one option and putting the others aside for later.

## Implementation

### Problem

As we can see below, the **Problem** can be represented in an abstract way and then used to build more specific problems without having to reimplement everything. As stated before to have a well defined problem we have the `initial` state, the `actions` listing and describing what the agent can do, the `goal_test` and the `path_cost`.

### Node

**Nodes** in a tree are a data structure from which the tree itself is constructed. Each node must have a parent, a state and various fields recording actions, costs, etc.

In [1]:
### Define a Problem class

class Problem:
    """This is an abstract class for problem definition"""
    
    def __init__(self, initial, goal=None):
        """Define the initial state and possibly a goal"""
        self.initial = initial
        self.goal = goal
        
    def actions(self, state):
        """Return the actions that can be executed 
        in the given state"""
        raise NotImplementedError
    
    def result(self, state, action):
        """Return the state resulted from the given action"""
        raise NotImplementedError
        
    def goal_test(self, state):
        """Return True if the state is a goal"""
        if isinstance(self.goal, list):
            return any(el is self.goal for el in state)
        else:
            return state == self.goal
        
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at
        state2 from state1 via action assuming cost c to get
        up to state1."""
        return c + 1
    
    def value(self, state):
        """For optimization problems, each state has a value"""
        raise NotImplementedError
        

### Define a Node class

class Node:
    """A node in a search tree: contains a pointer to the parent
    and to the actual state for this node. Also includes the
    action that got us to this state and the total path_cost."""
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a Node derived from a parent by an action"""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1
            
    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __lt__(self, node):
        return self.state < node.state
    
    def expand(self, problem):
        """List Nodes reachable in one step from this Node"""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]
    
    def child_node(self, problem, action):
        """Takes an action and returns the resulting child node"""
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next))
    
    def solution(self):
        """Return the sequence of actions to reach this Node"""
        return [node.action for node in self.path()[1:]]
    
    def path(self):
        """Return Nodes forming the path to reach this one"""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    # Methods avoid duplicated states, use them if needed
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state
    
    def __hash__(self):
        return hash(self.state)

# Uninformed Search Strategies

**Uninformed search** strategies work without any additional information about states beyond that provided in the problem definition: all they can do is to generate successors and distinguish between a goal state and a non-goal state.

Below we find a summary description of the main uninformed search strategies and their implementation.

## Breadth-first search

This is a simple strategy: the *root* node is expanded first, then all the successors of the root are expanded next, then their successors and so on.

**Breadth-first** is poised to find a solution eventually if given enough time and memory. For this algorithm the main issue is memory since it increases pretty fast.

## Uniform-cost search

Instead of expanding the shallowest node, **uniform-cost search** expands the node with the lowest path cost $g(n)$. This algorithm is optimal because it expands nodes in order of their optimal path cost.

## Depth-first search

Always expand the deepest node in the current *frontier* up to the node that has no successors. After expanding the nodes, they are dropped from the frontier, in this way search continues backing up to unexpanded nodes. By dropping nodes the memory requirements are much less than **breadth-first**.

## Depth-limited search

In infinite spaces **depth-first** will run forever or until it runs out of memory. To avoid this we can feed the algorithm with $l$, a predetermined depth at which the tree treats the nodes as having no successors. This is **depth-limited search**.

## Iterative deepening depth-first search

We usually use **iterative deepening** in combination with **depth-first**: we gradually increase the limit $l$ until a solution is found.

## Bidirectional search

The idea is to run 2 simultaneous searches: one from the root and one from the goal hoping that the searches meet in the middle. We do it by replacing the goal test with a check to see whether the frontiers of the two intersect.

In [2]:
# Parent functions
def tree_search(problem, frontier):
    """Search through the successors to find a goal.
    frontier should be an empty queue.
    Doesn't check for repeted paths to a state"""
    frontier.append(Node(problem.initial))
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None

def graph_search(problem, frontier):
    """Search through the successors to find a goal.
    frontier should be an empty queue.
    If two paths reach a state, use only the first"""
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child
                        not in frontier)
    return None

### Breadth-first
from collections import deque

class FIFO(deque):
    """Helper class to easily create and manage FIFO queues"""
    
    def pop(self):
        return self.popleft()

def breadth_first_tree_search(problem):
    """Search the shallowest nodes first"""
    return tree_search(problem, FIFO())

### Depth-first
def Stack():
    """Helper func to easily generate LIFO queues or Stacks"""
    return []

def depth_first_tree_search(problem):
    return tree_search(problem, Stack())

### Breadth-first

In [3]:
def breadth_first_search(problem):
    """Expand all nodes on the frontier until a solution is found"""
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = FIFO()
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

### Uniform-cost

For **uniform-cost search** it is efficient to previously define **Best-first search**. We will see it better later. <a id='uniform-cost'></a>

In [4]:
# Memoization needed for storing values
from functools import lru_cache
import bisect

def memoize(fn, slot=None, maxsize=32):
    """Memoize: remember the computed value for any argument
    list. If slot, store result in that slot. If not, use
    lru_cache for caching values"""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)
    
    return memoized_fn

class PriorityQueue(deque):
    """Helper class to easily create and manage a
    Priority Queue. The min or max element determined
    by f and order is returned first."""
    
    def __init__(self, order=min, f=lambda x: x):
        self.A = []
        self.order = order
        self.f = f
        
    def append(self, item):
        bisect.insort(self.A, (self.f(item), item))
        
    def __len__(self):
        return len(self.A)
    
    def pop(self):
        if self.order == min:
            return self.A.pop(0)[1]
        else:
            return self.A.pop()[1]
        
    def __contains__(self, item):
        return any(item == pair[1] for pair in self.A)
    
    def __getitem__(self, key):
        for _, item in self.A:
            if item == key:
                return item
            
    def __delitem__(self, key):
        for i, (value, item) in enumerate(self.A):
            if item == key:
                self.A.pop(i)

def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    The user specifies the function f(node) to min.
    If f is node.depth we have breadth-first. With 'memoize'
    we cache f values in the nodes as they are computed."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue(min, f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                incumbent = frontier[child]
                if f(child) < f(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
    return None

def uniform_cost_search(problem):
    """Expand nodes with the lowest g(n) first."""
    return best_first_graph_search(problem, 
                                   lambda node: node.path_cost)

### Depth-limited

In [5]:
def depth_limited_search(problem, limit=50):
    """Perform depth-first only up to a 
    limit depth"""
    def recursive_dls(node, problem, limit):
        if problem.goal_test(node.state):
            return node
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occured = False
            for child in node.expand(problem):
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occured = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occured else None
    return recursive_dls(Node(problem.initial), problem, limit)

### Iterative deepening

In [6]:
def iterative_deepening_search(problem):
    """Incrementally increase the limit until a
    solution is found"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result

# Informed (Heuristic) Search Strategies

**Informed search** using problem specific knowledge can find solutions more efficiently than **uninformed strategies**. The general approach is to use [**best-first search**](#uniform-cost): it is always based on tree or graph search, but the nodes are selected for expansion by using an **evaluation function** $f(n)$.

Most **best-first** algos include as a component of $f(n)$ a **heuristic function** denoted $h(n)$:

$$
h(n) = estimated\: cost\: of\: cheapest\: path\: from\: state\: in\: \textbf{n}\: to\: a\: goal
$$

## Greedy Best-first search

**Greedy best-first** tries to expand the node that is closest to the goal considering only $f(n)=h(n)$. This makes it try to optimize every step by getting as closer as possible to a solution.

## A\* search

**A\*** is the most known form of best-first search: it evaluates nodes by combining $g(n)$ - the cost to reach the node - and $h(n)$.

$$
f(n)=g(n)+h(n)
$$

This means that $f(n)$ represents the *estimated cost of the cheapest solution through n*. In this way **A\*** is both *complete* and *optimal* (while greedy best-first is neither). But to be complete and optimal we require 2 conditions:

- **Admissible heuristic** -> the heuristic must never overestimate the cost to reach the goal
- **Consistency** -> for every *n* and every successor *n'* generated by any action *a*, the estimated cost of reaching the goal from n is no greater than the step cost of getting to *n'* plus the estimated cost of reaching the goal from *n'* (required only by graph search) - basically we are ensuring that the triangle inequality holds true

**A\*** ensures optimality because everytime it selects a node for expansion does it through the optimal path to that node. This is because we have nondecreasing f-costs along any path, so to show how it works we can draw contours in the state space 

![contours](http://i.imgur.com/mBwiGMD.jpg)

Inside contours we are sure that every node will have the same or less $f(n)$ value. The nodes outside contours are not considered for expansion, so in the figure above we say that the subtree below *T* is **pruned**. Moreover, **A\*** is **optimally efficient**, meaning that no other algorithm of this type can expand fewer nodes than **A\***.